In [18]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os, sys
sys.path.append(os.path.join('..','..','libs'))
from read_smn import read_smn

# RNN en predicciones

Vamos a ver cómo utilizar RNNs para predecir datos futuros. En este caso utilizaremos los datos del SMN para poder comparar con lo estudiado de MLP

## Problema univariado

In [19]:
# leemos los datos
readr = read_smn(os.path.join('..','..','Data','junio-SMN','horario'))

In [20]:
# Utilizamos sólo SALTA
tstamps, data = readr.filter_by_station('SALTA')
data = data[:,0]  # solo la temperatura
train = data[:600] # split train y test
test = data[600:]

In [21]:
from sklearn.preprocessing import MinMaxScaler
class DataPipeline:
    def __init__(self, train_data, lookback) -> None:
        self.train_data = train_data.reshape(-1,1)
        self.scaler = MinMaxScaler()
        self.train_data = self.scaler.fit_transform(self.train_data)
        self.lookback = lookback
        self.train_x, self.train_y = self.split_series(self.train_data)

    def transform(self, data):
        '''wrapper to transform new data'''
        return self.scaler.transform(data)
    
    def inverse_transform(self, data):
        '''wraper to inverse transform'''
        return self.scaler.inverse_transform(data)
    
    def split_series(self, data):
        ''' tenemos una unica serie, entonces la partimos en lookback pedacitos
            para crear muchas series cortas de longitud lookback
        '''
        x = []
        y = []
        for i in range(self.train_data.shape[0]-self.lookback):
            x.append(self.train_data[i:i+self.lookback]) # creamos directamente los arrays con la forma que queremos
            y.append(self.train_data[i+self.lookback])
        return np.array(x), np.array(y)

    def perform_transformations(self, data):
        '''
        This method perform transformations for any single valued series
        series length must be larger than lookback
        '''
        assert len(data) > self.lookback
        data = self.transform(data)
        x, y = self.split_series(data)
        return x, y

In [22]:
# Usamos un dataset y dataloader para hacer todo mas simple con pytorch
# como las series ya traen la forma que necesitamos, 
# solo hay que hacer un cast de numpy a pytorch

class DataSet(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.X = x
        self.Y = y
        self.X = torch.from_numpy(self.X).float()
        self.Y = torch.from_numpy(self.Y).float()

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [37]:
# vamos a crear un objeto para preprocesar los datos
lookback = 12

pipe = DataPipeline(train_data= train, lookback= lookback)

# creamos los datasets y dataloaders en un diccionario para manejar mas simple

dataset = DataSet(pipe.train_x, pipe.train_y)

dataloader = torch.utils.data.DataLoader(dataset, batch_size = 64, shuffle= True)

print(f'Tenemos {len(dataset)} series temporales de largo {lookback}')

Tenemos 588 series temporales de largo 12


In [24]:
# Definimos la rnn mas pequeña posible
class RNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.rnn  = torch.nn.RNN(
                                input_size  = 1,    # la serie es univariada y entra una sola feature
                                hidden_size = 1,
                                num_layers  = 1,
                                batch_first = True
                                )
    
    def forward(self, x):
        x, h = self.rnn(x)
        return x, h     # retornamos todos los valores

rnn = RNN()
print(rnn)

RNN(
  (rnn): RNN(1, 1, batch_first=True)
)


In [25]:
print('parametros de la rnn')
print('-'*50)
print(f'weights: input-hidden -layer:0 = {rnn.rnn.weight_ih_l0}')
print(f'weights: hidden-hidden -layer:0 = {rnn.rnn.weight_hh_l0}')
print(f'bias: input-hidden -layer:0 = {rnn.rnn.bias_ih_l0}')
print(f'bias: hidden-hidden -layer:0 = {rnn.rnn.bias_hh_l0}')

parametros de la rnn
--------------------------------------------------
weights: input-hidden -layer:0 = Parameter containing:
tensor([[-0.5974]], requires_grad=True)
weights: hidden-hidden -layer:0 = Parameter containing:
tensor([[-0.7138]], requires_grad=True)
bias: input-hidden -layer:0 = Parameter containing:
tensor([0.2100], requires_grad=True)
bias: hidden-hidden -layer:0 = Parameter containing:
tensor([-0.7336], requires_grad=True)


In [26]:
# veamos como se comporta nuestra red super simple
x = np.ones(shape=(1,lookback,1))   # hacemos un vector de unos
x = torch.from_numpy(x).float()     # lo pasamos a 
y, h = rnn(x)
print('forward del modelo:') # una salida por cada paso temporal
print(y)
print('Estado de las capas ocultas al final:') # el estado de las capas ocultas al final de la serie
print(h)

forward del modelo:
tensor([[[-0.8079],
         [-0.4962],
         [-0.6450],
         [-0.5787],
         [-0.6093],
         [-0.5954],
         [-0.6018],
         [-0.5989],
         [-0.6002],
         [-0.5996],
         [-0.5999],
         [-0.5998]]], grad_fn=<TransposeBackward1>)
Estado de las capas ocultas al final:
tensor([[[-0.5998]]], grad_fn=<StackBackward0>)


In [27]:
def fit(model, dataloader, epochs = 10, eval = False):
    '''
    Funcion para entrenar el modelo model utilizando un dataloader
    '''
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)
    criterion = torch.nn.MSELoss()
    history = []
    for epoch in range(1,epochs+1):
        model.train()   # ponemos el modelo para ser entrenado
        train_h = [] 
        
        # leer los datos en el dataloader es muy simple (recorrer por batches)! 
        for x_b, y_b in dataloader['train']:

            # ponemos los gradientes a cero
            optimizer.zero_grad()
            
            y_pred = model(x_b)
            loss = criterion(y_pred, y_b)

            # calculamos los gradientes
            loss.backward()
            
            # actualizamos todos los pesos
            optimizer.step()
            train_h.append(loss.item())

        if eval:
            model.eval() # no estamos entrenando
            test_h = [] 
            with torch.no_grad():  # no vamos a hacer backward, solo ver la metrica sobre el test
                for x_b, y_b in dataloader['valid']:
                    y_pred = model(x_b)
                    loss = criterion(y_pred, y_b)
                    test_h.append(loss.item())
        if (epoch%2 == 0):
            if eval:
                print(f'epoch: {epoch}/{epochs} - train loss: {np.mean(train_h):.3f} - valid loss: {np.mean(test_h):.3f}')
                history.append([np.mean(train_h), np.mean(test_h)])
            else:
                print(f'epoch: {epoch}/{epochs} - train loss: {np.mean(train_h):.3f}')
                history.append([np.mean(train_h)])
    return history


In [28]:
# creamos un modelo que devuelva solo el ultimo valor (MANY TO ONE)
class RNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.rnn  = torch.nn.RNN(
                                input_size  = 1,    # la serie es univariada y entra una sola feature
                                hidden_size = 1,
                                num_layers  = 1,
                                batch_first = True
                                )
    
    def forward(self, x):
        x, _ = self.rnn(x)
        return x[:,-1]     # retornamos el último valor

In [30]:
rnn = RNN()
his = fit(rnn,{'train': dataloader})

epoch: 2/10 - train loss: 0.082
epoch: 4/10 - train loss: 0.067
epoch: 6/10 - train loss: 0.053
epoch: 8/10 - train loss: 0.043
epoch: 10/10 - train loss: 0.037
